# 4.10 Coding Etiquette and Excel Reporting (Part 1)

## Table of Contents

- Importing libraries & data

- Addressing Personally Identifiable Information (PII)

- Creating a regional flag within the dataset

- Excluding low-activity customers

## Importing libraries and Data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# create path
path = r'/Users/rusamijan/03-2023 Instacart Basket Analysis'

In [3]:
# import the update merged file
df_merged_all = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_customers_all.pkl'))

In [4]:
# check column name and number of columns
df_merged_all.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,spending_flag,median_prior_orders,order_frequency_flag,gender,state,age,date_of_registration,number_dependants,marital_status,income
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196.0,1.0,1.0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196.0,1.0,1.0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196.0,1.0,1.0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196.0,1.0,1.0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423


In [6]:
# Check data shape
df_merged_all.shape

(32404859, 31)

In [7]:
# check data types
df_merged_all.dtypes

order_id                     int64
user_id                      int64
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
product_id                 float64
add_to_cart_order          float64
reordered                  float64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
_merge                    category
price_range_loc             object
busiest_day                 object
busiest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
average_price              float64
spending_flag               object
median_prior_orders        float64
order_frequency_flag        object
gender                      object
state                       object
age                          int64
date_of_registration        object
number_dependants   

## Addressing Personally Identifiable Information (PII)

In our current merged data set we have no personal identification information (PPI). 

In our original 'customer' data set, we reomved columns: first name and surname. They are not necessary in our analysis and they would have fallen within the parameters of PPI.  

## Comparing customer behaviour across different regions. Determine whether there’s a difference in spending habits between the different U.S.regions

#### First, comparing customer behavior in different geographic areas across the US

In [10]:
# Checking state column
df_merged_all['state'].value_counts(dropna = False)

Pennsylvania            667082
California              659783
Rhode Island            656913
Georgia                 656389
New Mexico              654494
Arizona                 653964
North Carolina          651900
Oklahoma                651739
Alaska                  648495
Minnesota               647825
Massachusetts           646358
Wyoming                 644255
Virginia                641421
Missouri                640732
Texas                   640394
Colorado                639280
Maine                   638583
North Dakota            638491
Alabama                 638003
Kansas                  637538
Louisiana               637482
Delaware                637024
South Carolina          636754
Oregon                  636425
Arkansas                636144
Nevada                  636139
New York                635983
Montana                 635265
South Dakota            633772
Illinois                633024
Hawaii                  632901
Washington              632852
Mississi

In [11]:
# Defining region lists
region_1_NE = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
region_2_MW = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
region_3_S = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
region_4_W = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [15]:
# Assigning regions values to a new column (region)
df_merged_all.loc[df_merged_all['state'].isin(region_1_NE), 'region'] = 'Northeast'
df_merged_all.loc[df_merged_all['state'].isin(region_2_MW), 'region'] = 'Midwest'
df_merged_all.loc[df_merged_all['state'].isin(region_3_S), 'region'] = 'South'
df_merged_all.loc[df_merged_all['state'].isin(region_4_W), 'region'] = 'West'

In [16]:
# Check number rows or output in each region
df_merged_all['region'].value_counts(dropna = False)

South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: region, dtype: int64

In [17]:
# Check the data shape after adding a new column 'region'
df_merged_all.shape

(32404859, 32)

In [23]:
# Analysing spending by regions (using crosstab)
crosstab_spending_habits = pd.crosstab (df_merged_all['region'], df_merged_all['spending_flag'], dropna = False)

In [24]:
# Check output
crosstab_spending_habits

spending_flag,High spender,Low spender
region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


#### Second, creating an exclusion flag for low-activity customers (customers with less than 5 orders) and excluding them from the data. 

In [26]:
# Creating the exclusion flag 
df_merged_all.loc[df_merged_all['max_order'] < 5, 'activity_flag'] = 'Low-activity'

In [37]:
df_merged_all.loc[df_merged_all['max_order'] >= 5, 'activity_flag'] = 'High-activity'

In [32]:
df_merged_all.tail(10)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,order_frequency_flag,gender,state,age,date_of_registration,number_dependants,marital_status,income,region,activity_flag
32404849,1539810,106143,21,1,18,5.0,19675.0,1.0,1.0,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755,West,High-activity
32404850,3308056,106143,22,4,20,10.0,19675.0,1.0,1.0,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755,West,High-activity
32404851,2988973,106143,23,2,22,5.0,19675.0,1.0,1.0,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755,West,High-activity
32404852,930,106143,24,6,12,4.0,19675.0,1.0,1.0,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755,West,High-activity
32404853,467253,106143,25,6,16,7.0,19675.0,1.0,1.0,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755,West,High-activity
32404854,156685,106143,26,4,23,5.0,19675.0,1.0,1.0,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755,West,High-activity
32404855,484769,66343,1,6,11,NaN,47210.0,1.0,0.0,Fresh Farmed Tilapia Fillet,...,Non-frequent customer,Female,Tennessee,22,9/12/2017,3,married,46151,South,Low-activity
32404856,1561557,66343,2,1,11,30.0,47210.0,1.0,1.0,Fresh Farmed Tilapia Fillet,...,Non-frequent customer,Female,Tennessee,22,9/12/2017,3,married,46151,South,Low-activity
32404857,276317,66343,3,6,15,19.0,47210.0,1.0,1.0,Fresh Farmed Tilapia Fillet,...,Non-frequent customer,Female,Tennessee,22,9/12/2017,3,married,46151,South,Low-activity
32404858,2922475,66343,4,1,12,30.0,47210.0,1.0,1.0,Fresh Farmed Tilapia Fillet,...,Non-frequent customer,Female,Tennessee,22,9/12/2017,3,married,46151,South,Low-activity


In [28]:
#checking new added column
df_merged_all['activity_flag'].value_counts(dropna = False)

High-activity    30964564
Low-activity      1440295
Name: activity_flag, dtype: int64

In [29]:
df_merged_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 33 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   order_number            int64   
 3   orders_day_of_week      int64   
 4   order_hour_of_day       int64   
 5   days_since_prior_order  float64 
 6   product_id              float64 
 7   add_to_cart_order       float64 
 8   reordered               float64 
 9   product_name            object  
 10  aisle_id                int64   
 11  department_id           int64   
 12  prices                  float64 
 13  _merge                  category
 14  price_range_loc         object  
 15  busiest_day             object  
 16  busiest_days            object  
 17  busiest_period_of_day   object  
 18  max_order               int64   
 19  loyalty_flag            object  
 20  average_price           float64 
 21  spendi

In [46]:
#creating a new dataframe minus low-activity customers
df_high_activity = df_merged_all.loc[df_merged_all['activity_flag'] == 'High-activity']

In [50]:
#creating a new dataframe minus high-activity customers
df_low_activity = df_merged_all.loc[df_merged_all['activity_flag'] == 'Low-activity']

In [47]:
df_high_activity.head(10)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,order_frequency_flag,gender,state,age,date_of_registration,number_dependants,marital_status,income,region,activity_flag
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,High-activity
1,2398795,1,2,3,7,15.0,196.0,1.0,1.0,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,High-activity
2,473747,1,3,3,12,21.0,196.0,1.0,1.0,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,High-activity
3,2254736,1,4,4,7,29.0,196.0,1.0,1.0,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,High-activity
4,431534,1,5,4,15,28.0,196.0,1.0,1.0,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,High-activity
5,3367565,1,6,2,7,19.0,196.0,1.0,1.0,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,High-activity
6,550135,1,7,1,9,20.0,196.0,1.0,1.0,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,High-activity
7,3108588,1,8,1,14,14.0,196.0,2.0,1.0,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,High-activity
8,2295261,1,9,1,16,0.0,196.0,4.0,1.0,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,High-activity
9,2550362,1,10,4,8,30.0,196.0,1.0,1.0,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,High-activity


In [51]:
df_low_activity.head(10)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,order_frequency_flag,gender,state,age,date_of_registration,number_dependants,marital_status,income,region,activity_flag
1510,520620,120,1,3,11,NaN,196.0,2.0,0.0,Soda,...,Regular customer,Female,Kentucky,54,3/2/2017,2,married,99219,South,Low-activity
1511,3273029,120,3,2,8,19.0,196.0,2.0,1.0,Soda,...,Regular customer,Female,Kentucky,54,3/2/2017,2,married,99219,South,Low-activity
1512,520620,120,1,3,11,NaN,46149.0,1.0,0.0,Zero Calorie Cola,...,Regular customer,Female,Kentucky,54,3/2/2017,2,married,99219,South,Low-activity
1513,3273029,120,3,2,8,19.0,46149.0,1.0,1.0,Zero Calorie Cola,...,Regular customer,Female,Kentucky,54,3/2/2017,2,married,99219,South,Low-activity
1514,520620,120,1,3,11,NaN,26348.0,3.0,0.0,Mixed Fruit Fruit Snacks,...,Regular customer,Female,Kentucky,54,3/2/2017,2,married,99219,South,Low-activity
1515,906054,120,2,4,7,15.0,31102.0,1.0,0.0,Sugar Free Energy Drink,...,Regular customer,Female,Kentucky,54,3/2/2017,2,married,99219,South,Low-activity
1516,3273029,120,3,2,8,19.0,31102.0,3.0,1.0,Sugar Free Energy Drink,...,Regular customer,Female,Kentucky,54,3/2/2017,2,married,99219,South,Low-activity
3855,3226575,360,1,5,12,NaN,196.0,1.0,0.0,Soda,...,Frequent customer,Male,Arizona,46,9/1/2017,1,married,104257,West,Low-activity
3856,3046940,360,2,2,8,4.0,5322.0,3.0,0.0,Gluten Free Dark Chocolate Chunk Chewy with a ...,...,Frequent customer,Male,Arizona,46,9/1/2017,1,married,104257,West,Low-activity
3857,2300993,360,3,1,13,6.0,5322.0,2.0,1.0,Gluten Free Dark Chocolate Chunk Chewy with a ...,...,Frequent customer,Male,Arizona,46,9/1/2017,1,married,104257,West,Low-activity


In [54]:
df_high_activity.shape

(30964564, 33)

In [ ]:
df_low_activity.shape

(1440295, 33)

In [52]:
# Export high_activity in .pkl format 
df_high_activity.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'high_activity_customers.pkl'))

In [53]:
# Export low_activity in .pkl format 
df_low_activity.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'low_activity_customers.pkl'))